# DATASCI W261: Machine Learning at Scale
## Assignment Week 7
Miki Seltzer (miki.seltzer@berkeley.edu)<br>
W261-2, Spring 2016<br>
Submission: 

In [1]:
# We will need these so we can reload modules as we modify them
%load_ext autoreload
%autoreload 2

# HW 7.0: Shortest path graph distances (toy networks)

In this part of your assignment you will develop the base of your code for the week.

Write MRJob classes to find shortest path graph distances, as described in the lectures. In addition to finding the distances, your code should also output a distance-minimizing path between the source and target. Work locally for this part of the assignment, and use both of the undirected and directed toy networks.

![Toy networks](toy_graphs.png)

To proof you code's function, run the following jobs

- shortest path in the undirected network from node 1 to node 4
Solution: 1,5,4 

- shortest path in the directed network from node 1 to node 5
Solution: 1,2,4,5

and report your output -- make sure it is correct!

## Initialize graph structure

We have the graph encoded as an adjacency list, but we need to keep track of state in each iteration
- shortest distance from start node
- node state (unvisited, queued, visited)
- path taken to get to node

In [3]:
%%writefile MRJob_Initiate.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import sys

class initiate(MRJob):
        
    # Specify some custom options so we only have to write one MRJob class for each join
    def configure_options(self):
        super(initiate, self).configure_options()
        self.add_passthrough_option('--startNode', default='1')
        
    def mapper(self, _, line):
        fields = line.strip().split('\t')
        name = fields[0]
        neighbors = eval(fields[1])
        if name == self.options.startNode:
            yield name, [neighbors, 0, 'Q', [name]]
        else:
            yield name, [neighbors, sys.maxint, 'U', []]
        
if __name__ == '__main__':
    initiate.run()

Overwriting MRJob_Initiate.py


## Iterate through graph structure

In each mapper iteration:
- Expand each node that is in queued state, then mark that node as visited

In each reducer iteration:
- If any record for a node has a visited state, emit the visited record
- When we keep track of state, if a record has a state of queued, then the node needs to be merged
- If the record is truly unvisited, emit the unvisited node

In [31]:
%%writefile MRJob_ShortestPath.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import sys

class shortestPath(MRJob):
    
    """
    Mapper: Iterate over each node in graph file
    - Expand frontier if needed
    - Update node statuses
    """
    def mapper(self, _, line):

        # Split text to get our data
        fields = line.strip().split('\t')
        
        # If running locally, don't need to eval
        #name = fields[0]
        
        # If using EMR, need to eval the string
        name = eval(fields[0])
        value = eval(fields[1])
        neighbors = value[0]
        distance = int(value[1])
        status = value[2]
        path = value[3]
        
        # If this node is queued, expand the frontier
        #  - mark current node as visited
        #  - yield neighbor nodes into queue
        if status == 'Q':
            yield name, [neighbors, distance, 'V', path]
            if neighbors:
                for node in neighbors:
                    temp_path = list(path)
                    temp_path.append(node)
                    yield node, [None, distance + 1, 'Q', temp_path]
        else:
            yield name, [neighbors, distance, status, path]
        
        
    """
    Reducer: Aggregate expanded nodes
    """
    def reducer(self, key, values):
        neighbors = {}
        distance = sys.maxint
        status = None
        path = []
        
        for val in values:
            
            # We've hit a visited node. Break out of the loop.
            if val[2] == 'V':
                neighbors = val[0]
                distance = val[1]
                status = val[2]
                path = val[3]
                break
            
            # We've hit an unvisited node. Collect the neighbors and the status
            # If status is already Q, do not overwrite
            elif val[0]: 
                neighbors = val[0]
                if status != 'Q':
                    status = val[2]
            
            # We've hit a queued node. Update status and path
            else:
                status = val[2]
                path = val[3]
                
            # Update minimum distance if necessary
            distance = min(distance, val[1])
            
        yield key, [neighbors, distance, status, path]
    
if __name__ == '__main__':
    shortestPath.run()

Overwriting MRJob_ShortestPath.py


In [15]:
from MRJob_Initiate import initiate
from MRJob_ShortestPath import shortestPath

def findShortestPath(filename, startNode, endNode):

    filenameState = filename.replace('.', '_state.')
    
    # Initiate graph adjacency list to track state
    mr_job_init = initiate(args=[filename, '--no-strict-protocols', '--startNode', startNode])
        
    myfile = open(filenameState, 'w')
    
    with mr_job_init.make_runner() as runner:
        runner.run()
        
        for line in runner.stream_output():
            out = mr_job_init.parse_output_line(line)
            myfile.write(str(out[0]) + '\t' + str(out[1]) + '\n')

    myfile.close()
    
    # Iterate over the adjacency list with state until all nodes are visited    
    filenameTemp = filenameState.replace('.', 'Temp.')
    
    mr_job = shortestPath(args=[filenameState, '--no-strict-protocols'])

    visitedEndNode = False
    
    while not visitedEndNode:

        myfile = open(filenameTemp, 'w')
        
        with mr_job.make_runner() as runner: 
            # Run MRJob
            runner.run()

            # Write stream_output to file
            for line in runner.stream_output():
                out = mr_job.parse_output_line(line)
                myfile.write(str(out[0]) + '\t' + str(out[1]) + '\n')
                if out[0] == endNode and out[1][2] == 'V':
                    visitedEndNode = True
                    path = out[1][3]
                    return path
                
        myfile.close()
        !mv {filenameTemp} {filenameState}

filename = 'undirected_toy.txt'
print 'Shortest path in', filename
path = findShortestPath(filename, '1', '4')
print path

filename = 'directed_toy.txt'
print '\nShortest path in', filename
path = findShortestPath('directed_toy.txt', '1', '5')
print path

Shortest path in undirected_toy.txt
['1', '5', '4']

Shortest path in directed_toy.txt
['1', '2', '4', '5']


# HW 7.1: Exploratory data analysis (NLTK synonyms)

For the NLTK data set, find:
- Number of nodes
- Number of links
- Average degree

In [217]:
%%writefile MRJob_Explore.py
from mrjob.job import MRJob
from mrjob.step import MRStep

class explore(MRJob):
        
    # Specify some custom options so we only have to write one MRJob class for each join
    def configure_options(self):
        super(explore, self).configure_options()
        self.add_passthrough_option('--exploreType', default='nodes')
        
    """
    Find number of nodes
    """
    def mapper_discoverNodes(self, _, line):
        fields = line.strip().split('\t')
        name = fields[0]
        neighbors = eval(fields[1])
        yield name, 1
        if neighbors:
            for node in neighbors:
                yield node, 1
        
    def reducer_discoverNodes(self, key, values):
        yield key, 1
        
    def mapper_countNodes(self, key, value):
        yield None, 1
        
    def reducer_countNodes(self, key, values):
        yield None, sum(values)
    
    """
    Find number of links
    """
    def mapper_links(self, _, line):
        fields = line.strip().split('\t')
        name = fields[0]
        neighbors = eval(fields[1])
        if neighbors:
            for node in neighbors:
                yield None, 1
        
    def reducer_links(self, key, values):
        yield None, sum(values)
    
    """
    Find average in-degree and out-degree
    """
    
    # For the mapper, we need to emit number of out and in links
    # Count number of neighbors to find num links going out
    # Count 1 for each node in neighbor list to find num links coming in
    # Yield in the form of nodeName, (inLinks, outLinks)
    def mapper_degrees(self, _, line):
        fields = line.strip().split('\t')
        name = fields[0]
        neighbors = eval(fields[1])
        
        # Yield number of links going out
        yield name, (0, len(neighbors))
        
        # Yield 1 link coming in for each node in neighbors
        if neighbors:
            for node in neighbors:
                yield node, (1, 0)
            
    def reducer_degrees(self, key, values):
        inSum = 0
        outSum = 0
        for val in values:
            inSum += val[0]
            outSum += val[1]
        yield key, (inSum, outSum)
        
    def mapper_degreesAvg(self, key, value):
        yield None, (value[0], value[1], 1)
        
    def combiner_degreesAvg(self, key, values):
        count = 0
        inSum = 0
        outSum = 0
        for val in values:
            inSum += val[0]
            outSum += val[1]
            count += val[2]
        yield None, (inSum, outSum, count)
        
    def reducer_degreesAvg(self, key, values):
        count = 0.0
        inSum = 0.0
        outSum = 0.0
        for val in values:
            inSum += val[0]
            outSum += val[1]
            count += val[2]
        yield None, (inSum / count, outSum / count)
        
    """
    Multi-step pipeline
    """
    def steps(self):
        if self.options.exploreType == 'nodes':
            return [
                MRStep(mapper=self.mapper_discoverNodes,
                       combiner=self.reducer_discoverNodes,
                       reducer=self.reducer_discoverNodes),
                MRStep(mapper=self.mapper_countNodes,
                       combiner=self.reducer_countNodes,
                       reducer=self.reducer_countNodes)
            ]
        elif self.options.exploreType == 'links':
            return [
                MRStep(mapper=self.mapper_links,
                       combiner=self.reducer_links,
                       reducer=self.reducer_links)
            ]
        elif self.options.exploreType == 'degrees':
            return [
                MRStep(mapper=self.mapper_degrees,
                       combiner=self.reducer_degrees,
                       reducer=self.reducer_degrees),
                MRStep(mapper=self.mapper_degreesAvg,
                       combiner=self.combiner_degreesAvg,
                       reducer=self.reducer_degreesAvg)
            ]

        
if __name__ == '__main__':
    explore.run()

Overwriting MRJob_Explore.py


In [218]:
from MRJob_Explore import explore

def exploreData(filename, exploreType):

    mr_job = explore(args=[filename, '--no-strict-protocols', '--exploreType', exploreType])
    output = []
    
    with mr_job.make_runner() as runner:
        runner.run()
        
        for line in runner.stream_output():
            out = mr_job.parse_output_line(line)
            if exploreType == 'nodes' or exploreType == 'links':
                print 'Number of', exploreType, '=', '{:,d}'.format(out[1])
            else:
                print 'Average in-links  =', '{:,.2f}'.format(out[1][0])
                print 'Average out-links =', '{:,.2f}'.format(out[1][1])

## Test exploreData on toy data sets

Using these graphs, test our job above

![Toy networks](toy_graphs.png)

Undirected graph has:
- 5 nodes
- 7 edges (bidirectional, so 14 links)
- 14 in and out links total = 14/5 average in and out links

Directed graph has:
- 6 nodes
- 12 links
- average of 2 incoming and outgoing links per node

In [219]:
print 'Undirected toy data'
exploreData('undirected_toy.txt', 'nodes')
exploreData('undirected_toy.txt', 'links')
exploreData('undirected_toy.txt', 'degrees')

print '\nDirected toy data'
exploreData('directed_toy.txt', 'nodes')
exploreData('directed_toy.txt', 'links')
exploreData('directed_toy.txt', 'degrees')

Undirected toy data
Number of nodes = 5
Number of links = 14
Average in-links  = 2.80
Average out-links = 2.80

Directed toy data
Number of nodes = 6
Number of links = 12
Average in-links  = 2.00
Average out-links = 2.00


In [220]:
print 'NLTK data'            
exploreData('synNet/synNet.txt', 'nodes')
exploreData('synNet/synNet.txt', 'links')
exploreData('synNet/synNet.txt', 'degrees')

NLTK data
Number of nodes = 8,271
Number of links = 61,134
Average in-links  = 7.39
Average out-links = 7.39


# HW 7.2: Shortest path graph distances (NLTK synonyms)

Find distance from "walk" (index 7827) to "make" (index 536).

In [12]:
def printPath(indexFile, path):
    myDict = {}

    with open(indexFile, 'r') as myfile:
        for line in myfile:
            fields = line.strip().split('\t')
            if fields[1] in path:
                myDict[fields[1]] = fields[0]

    print '{:<10s}{:<50s}'.format('INDEX', 'NAME')
    for node in path:
        print '{:<10s}{:<50s}'.format(node, myDict[node])

In [16]:
filename = 'synNet/synNet.txt'
path = findShortestPath(filename, '7827', '536')
printPath('synNet/indices.txt', path)

INDEX     NAME                                              
7827      walk                                              
4655      passes                                            
631       draw                                              
536       make                                              


## Setup job on EMR

We will need to modify our driver to use S3 buckets instead of storing locally

In [17]:
# Create job flow so that we don't need to keep spinning up clusters
!python -m mrjob.tools.emr.create_job_flow

using configs in /etc/mrjob.conf
using existing scratch bucket mrjob-ac40f1afcc0b86ce
using s3://mrjob-ac40f1afcc0b86ce/tmp/ as our scratch dir on S3
Creating persistent job flow to run several jobs in...
creating tmp directory /tmp/no_script.cloudera.20160306.182759.293738
writing master bootstrap script to /tmp/no_script.cloudera.20160306.182759.293738/b.py
Copying non-input files into s3://mrjob-ac40f1afcc0b86ce/tmp/no_script.cloudera.20160306.182759.293738/files/
Waiting 5.0s for S3 eventual consistency
Creating Elastic MapReduce job flow
Job flow created with ID: j-1TFS8W6PX3DIQ
j-1TFS8W6PX3DIQ


In [18]:
clusterId = 'j-1TFS8W6PX3DIQ'

In [33]:
from MRJob_Initiate import initiate
from MRJob_ShortestPath import shortestPath

def findShortestPathEMR(filename, startNode, endNode, clusterId):

    # Set the name of our output folder, and make sure it does not already exist
    outputDirState = '/'.join(['s3://ms-w261-hw07', filename.replace('.txt', '_state2').split('/')[-1]])
#     !aws s3 rm --recursive --quiet {outputDirState + '/'}
    
#     # Initiate graph adjacency list to track state
#     mr_job_init = initiate(args=[filename, '--no-strict-protocols', '--startNode', startNode,
#                                  '-r', 'emr', '--emr-job-flow-id', clusterId,
#                                  '--no-output', '--output-dir', outputDirState])
    
#     with mr_job_init.make_runner() as runner:
#         runner.run()
    
    # Iterate over the adjacency list with state until all nodes are visited    
    inputDir = outputDirState + '/'
    outputDir = outputDirState + 'Temp1'

    visitedEndNode = False
    i = 1

    while not visitedEndNode:
        
        mr_job = shortestPath(args=[inputDir, '--no-strict-protocols',
                                    '-r', 'emr', '--emr-job-flow-id', clusterId,
                                    '--output-dir', outputDir])
        
        with mr_job.make_runner() as runner: 
            # Run MRJob
            runner.run()

            # Write stream_output to file
            for line in runner.stream_output():
                out = mr_job.parse_output_line(line)
                if out[0] == endNode and out[1][2] != 'U':
                    visitedEndNode = True
                    return out[1][3]
                    
        # Update inputDir and outputDir for next iteration
        inputDir = outputDirState + 'Temp' + str(i) + '/'
        outputDir = outputDirState + 'Temp' + str(i + 1)
        
        i += 1


In [302]:
# Test on small data set
filename = 'synNet/synNet.txt'
startNode = '7827'
endNode = '631'

print filename
path = findShortestPathEMR(filename, startNode, endNode, clusterId)
# path = ['7827', '4655', '631']
print 'Shortest path from', startNode, 'to', endNode, '=', path

synNet/synNet.txt
Shortest path from 7827 to 631 = ['7827', '4655', '631']


# HW 7.3: Exploratory data analysis (Wikipedia)

Using MRJob, explore the Wikipedia network data on the AWS cloud. Reuse your code from HW 7.1 -- does is scale well? Be cautioned that Wikipedia is a directed network, where links are not symmetric. So, even though a node may be linked to, it will not appear as a primary record itself if it has no out-links. This means that you may have to ADJUST your code (depending on its design). To be sure of your code's functionality in this context, run a systems test on the directed_toy.txt network.

We will need to modify our driver to use EMR.

In [269]:
from MRJob_Explore import explore

def exploreDataEMR(filename, exploreType, clusterId):

    mr_job = explore(args=[filename, '--no-strict-protocols', '--exploreType', exploreType,
                           '-r', 'emr', '--emr-job-flow-id', clusterId,])
    output = []
    
    with mr_job.make_runner() as runner:
        runner.run()
        
        for line in runner.stream_output():
            out = mr_job.parse_output_line(line)
            if exploreType == 'nodes' or exploreType == 'links':
                print 'Number of', exploreType, '=', '{:,d}'.format(out[1])
            else:
                print 'Average in-links  =', '{:,.2f}'.format(out[1][0])
                print 'Average out-links =', '{:,.2f}'.format(out[1][1])

In [272]:
exploreDataEMR('s3://ucb-mids-mls-networks/wikipedia/all-pages-indexed-out.txt', 'nodes', clusterId)

Number of nodes = 15,192,277


In [273]:
exploreDataEMR('s3://ucb-mids-mls-networks/wikipedia/all-pages-indexed-out.txt', 'links', clusterId)

Number of links = 142,114,057


In [275]:
exploreDataEMR('s3://ucb-mids-mls-networks/wikipedia/all-pages-indexed-out.txt', 'degrees', clusterId)

Average in-links  = 54.96
Average out-links = 54.96


# HW 7.4: Shortest path graph distances (Wikipedia)

Using MRJob, find shortest path graph distances in the Wikipedia network on the AWS cloud. Reuse your code from 7.2, but once again be warned of Wikipedia being a directed network. To be sure of your code's functionality in this context, run a systems test on the directed_toy.txt network.

When running your code on the Wikipedia network, proof its function by running the job:

- shortest path from "Ireland" (index=6176135) to "University of California, Berkeley" (index=13466359),

and show your code's output.

Once your code is running, find some other shortest paths and report your results.

In [305]:
filename = 's3://ucb-mids-mls-networks/wikipedia/all-pages-indexed-out.txt'
path = findShortestPathEMR(filename, '6176135', '13466359', clusterId)

Input:  s3://ms-w261-hw07/all-pages-indexed-out_state/
Output: s3://ms-w261-hw07/all-pages-indexed-out_stateTemp1


Input: 

 s3://ms-w261-hw07/all-pages-indexed-out_stateTemp1/
Output: s3://ms-w261-hw07/all-pages-indexed-out_stateTemp2




In [315]:
printPath('wikipedia/indices.txt', path)

INDEX     NAME                                              
6176135   Ireland                                           
11607791  Seamus Heaney                                     
13466359  University of California, Berkeley                


In [34]:
filename = 's3://ucb-mids-mls-networks/wikipedia/all-pages-indexed-out.txt'

# Find distance between:
# - 'Margo Seltzer' (my aunt, prof of Computer Science)
# - 'Yale University' (my alma mater)
path = findShortestPathEMR(filename, '8375315', '14181325', clusterId)

printPath('wikipedia/indices.txt', path)

INDEX     NAME                                              
8375315   Margo Seltzer                                     
8730327   Michael Stonebraker                               
14181325  Yale University                                   


# HW 7.5: Conceptual exercise: Largest single-source network distances

#### Suppose you wanted to find the largest network distance from a single source, i.e., a node that is the furthest (but still reachable) from a single source.

#### How would you implement this task?
I would traverse the entire graph in a similar fashion to when we find the shortest path. Instead of initializing the distances from the source node to infinity, we initialize to zero. Then when a new node is discovered, we compare distances and choose the larger.

#### How is this different from finding the shortest path graph distances?
This is different because we MUST traverse the entire graph to find the longest distance. With shortest distance between a source and a target, we can stop once the target is reached. Additionally, the longest distance requires that the graph be a directed acyclic graph. If there are cycles in the graph, then there is no longest path, since we can just traverse a cycle inifinite times. We should also restrict our search to simple paths, where no node is repeated.

#### Is this task more difficult to implement than the shortest path distance? As you respond, please comment on program structure, runtimes, iterations, general system requirements, etc...
This task is more difficult to implement, and is in fact NP-hard if the graph is not a directed acyclic graph. 

In [35]:
# Terminate job flow so we don't rack up AWS expenses
!python -m mrjob.tools.emr.terminate_job_flow {clusterId}

using configs in /etc/mrjob.conf
using existing scratch bucket mrjob-ac40f1afcc0b86ce
using s3://mrjob-ac40f1afcc0b86ce/tmp/ as our scratch dir on S3
Terminated job flow j-1TFS8W6PX3DIQ
